In [1]:
import pandas as pd 
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import sqlite3 as sql 
import json

### Data gadharing by clling csv file which is being creared from the Text_chunking file that cretes and store the chunks of the text 

In [2]:
data = pd.read_csv(r'File_content.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'File_content.csv'

###  Performing NLTK taks
* TOkenize the chunk using the word tokinizer
* Removing the stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')

#### implementing word Tokinizer and sentence tokinizer ,lowering the sentence and removing punctuation 

In [ ]:
def remove_punctuation(text):
    return "".join(char for char in text if char not in string.punctuation)
def task_1 (text):
    try:
        # converting to lowercase
        text = text.lower()
        text = remove_punctuation(text)
        # ccreating word tokens 
        stop_words = set(stopwords.words('english'))
        word_token = sent_tokenize(text,language='english') 
        clean_word =  [words for words in word_token if words  not in stop_words]

        return "".join(clean_word)
    except Exception as e:
        return f"The error is {e}"
cleaned_data = [task_1(i) for i in data['chunk']]
print(cleaned_data)


In [ ]:
cleaned_data

In [ ]:
# lower_s_chunk=sample_chunk.lower()

In [ ]:
# stop_words = set(stopwords.words('english'))
# print(stop_words,end="")

In [ ]:
# word_token = word_tokenize(lower_s_chunk,language='english')

# # stopwords removal from the word_tokne 
# clean_data =  [i for i in word_token if i not in stop_words]


# print(clean_data)
# print(clean_data.__len__())

In [ ]:
# # sentence tokinization 

# sentence_tokinize =  sent_tokenize(lower_s_chunk)


# print(sentence_tokinize)

### Implementing Tf-Idf Vactorization 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
tf_idf_vactor = TfidfVectorizer()
tf_idf_vactor.fit(cleaned_data)
tfidf_data = tf_idf_vactor.transform(cleaned_data).toarray()
# print(arrayed_data)
# print(tf_idf_vactor.get_feature_names_out())

In [ ]:
tfidf_data

In [ ]:
pd.DataFrame(cleaned_data)

In [ ]:
tf_idf_vactor.get_feature_names_out()

In [ ]:
df = pd.DataFrame(tfidf_data,columns=tf_idf_vactor.get_feature_names_out())
df['full_sentence'] = cleaned_data
df.shape

### store in sqlLite 

In [ ]:
db_name = sql.connect('app_maindb.db')
cus = db_name.cursor()

In [ ]:
cus.execute("DROP TABLE IF EXISTS CHUNKS_TFIDF")

In [ ]:
table_creation_query = """ 
CREATE TABLE CHUNKS_TFIDF(
id INTEGER PRIMARY KEY AUTOINCREMENT,
chunk_txt TEXT NOT NULL , 
tfidf_vector REAL NOT NULL
)
"""

In [ ]:
cus.execute(table_creation_query)


In [ ]:
for cleaned_text ,vactors in zip(cleaned_data,tfidf_data):
    vector_json = json.dumps(vactors.tolist())

    cus.execute("INSERT INTO CHUNKS_TFIDF (chunk_txt, tfidf_vector) VALUES (?,?)",(cleaned_text, vector_json))
    # print(cleaned_text,vector_json)    


print(f"Successfully inserted {len(cleaned_data)} records into '{db_name}'.")
cus.execute("SELECT * FROM CHUNKS_TFIDF")
for row in cus.fetchall():
        print(row,end='')

db_name.commit()
# db_name.close()

In [ ]:
for text , vec in zip(tf_idf_vactor.get_feature_names_out(),tf_idf_vactor.idf_):
    print(text, vec)

In [ ]:
print(tf_idf_vactor.idf_)